# How to use this tutorial
All hyperlinks are [clickable](img/IMG1.jpg "What happened to the curious cat?"), text in <span title="This tutorial was written in Jupyter (iPython notebook) and rendered on Github."><b>bold</b></span> can have hovertext with additional information, and all code can be run in a Unix shell (Bash) terminal. 

# Introduction
<span title="The process of identifying the order of the individual components in a chain of molecules eg. nucleotides in RNA/DNA or amino acids in proteins."><b>Sequencing</b></span> plays an important role in diverse fields including *but not limited to* forensics, evolutionary biology, pharmacology, ecology, and oncology. Nucleic acid sequencing is not exactly a *new* technology; scientists have been using various methods of nucleotide sequencing since the 1970s. [Click here](http://www.sciencedirect.com/science/article/pii/S0888754315300410) to learn more about the history sequencing technology.

Have you ever worked with sequencing data before? If your answer was 'Yes', what were you trying to find? How was the data organized? Where did it come from? If your answer was 'No', get ready to try something new!

In this tutorial we are going to be working with the model organism *Saccharomyces cerevisiae*. As *S. cerevisiae* is a very well-studied organism, <span title="This isn't always a good idea; depending on your research objectives, you may need to consider which criteria were used to decide which features go in the reference annotations and what was discarded."><b>we will assume</b></span> that the reference genome & assembly are mostly correct and complete.

What kind of organism is *S. cerevisiae*? *S. cerevisiae* is a budding yeast; a <span title="It is simple in that yeast is among the easiest organisms to grow in a laboratory setting. It is only biologically simple relative to other multicellular eukaryotes like H. sapiens"><b>simple</b></span> eukaryote with 16 chromosomes and <span title="According to yeastgenome.org, there are 5155 'validated' ORFs. According to the NCBI, there are 6350 genes."><b>~6000 genes</b></span>. 
## Next Generation Sequencing (NGS)
Next generation sequencing [also known as  


## Methods
First you need to download and install the following programs:
* [FastQC 0.11.5 for Linux](http://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.5.zip) or [FastQC 0.11.5 for OSX (Mac)](http://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.5.dmg)
* [Trimmomatic 0.67](http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.36.zip)
* [Bowtie2 2.2.9 for Linux](https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.2.9/bowtie2-2.2.9-linux-x86_64.zip/download) or [Bowtie2 2.2.9 for OSX (Mac)](https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.2.9/bowtie2-2.2.9-macos-x86_64.zip/download)
* [Cufflinks 2.2.1 for Linux](http://cole-trapnell-lab.github.io/cufflinks/assets/downloads/cufflinks-2.2.1.Linux_x86_64.tar.gz)  or [Cufflinks 2.2.1 for OSX (Mac)](http://cole-trapnell-lab.github.io/cufflinks/assets/downloads/cufflinks-2.2.1.OSX_x86_64.tar.gz)  


## Potential problems and pitfalls





<span title=""><b> </b></span>

# 1. Download your reference genome and annotations

In [8]:
%%bash
#Download the reference genome and annotations for Saccharomyces cerevisiae (S288c)
wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz 
wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz
#Unzip the files
gunzip GCF_000146045.2_R64_genomic*
#Rename the files with names are easier to remember
mv GCF_000146045.2_R64_genomic.fna s_cerevisiae.fasta
mv GCF_000146045.2_R64_genomic.gff s_cerevisiae.gff

--2016-09-05 15:59:17--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz
           => “GCF_000146045.2_R64_genomic.fna.gz”
Resolving ftp.ncbi.nlm.nih.gov... 130.14.250.13, 2607:f220:41e:250::12
Connecting to ftp.ncbi.nlm.nih.gov|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCF_000146045.2_R64 ... done.
==> SIZE GCF_000146045.2_R64_genomic.fna.gz ... 3843463
==> PASV ... done.    ==> RETR GCF_000146045.2_R64_genomic.fna.gz ... done.
Length: 3843463 (3.7M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  1%  153K 24s
    50K .......... .......... .......... .......... ..........  2%  388K 17s
   100K .......... .......... .......... .......... ..........  3%  388K 14s
   150K .......... .......... .......... .......... ..........  5%  389K 13s
   200K .......... .......... .......... .......... ..

Due to the time constraints, we're not going to do our analysis on the whole genome; instead, we will work exclusively with chomosome 10 (ChrX). The Fasta file *s_cerevisiae.fasta* contains the sequences of all 16 chromosomes, but we only need chromosome 10. <span title="A lazy way to do this is to open the file s_cerevisiae.fasta in a text editor and search for 'NC_001142.9' which is the ID that corresponds to chromosome 10. Delete everything except for chromosome 10 then save the file as 's_cerevisiae_chrX.fasta'."><b>How would you make a new file called *s_cerevisiae_chrX.fasta* that only contains chromosome 10?</b></span> Here is a table which will help you figure out which header+sequence to keep:

| ID in s_cerevisiae.fasta | Chromosome |
|---------------------|------------|
| NC_001133.9         | I          |
| NC_001134.8         | II         |
| NC_001135.5         | III        |
| NC_001136.10        | IV         |
| NC_001137.3         | V          |
| NC_001138.5         | VI         |
| NC_001139.9         | VII        |
| NC_001140.6         | VIII       |
| NC_001141.2         | IX         |
| NC_001142.9         | X          |
| NC_001143.9         | XI         |
| NC_001144.5         | XII        |
| NC_001145.3         | XIII       |
| NC_001146.8         | XIV        |
| NC_001147.6         | XV         |
| NC_001148.4         | XVI        |
| NC_001224.1         | Mito       |

In [9]:
%%bash
### another way to make s_cerevisiae_chrX.fasta
head s_cerevisiae.fasta -n 82242 |tail -n 9323 > s_cerevisiae_chrX.fasta

We have to do the same thing with the annotations. We only want features which appear on chromosome X. How would you go about generating a new file, *s_cerevisiae_chrX.gff*, <span title="Remember, chromosome X is called 'NC_001142.9' in the annotations. Look at the GFF file- the first column of every line is the chromosome."><b> with only the features from chromosome 10?</b></span>

In [28]:
%%bash
grep "^NC_001142.9" s_cerevisiae.gff > s_cerevisiae_chrX.gff

# 2. Quality control with *FastQC*

# 3. Trimming RNAseq reads with *Trimmomatic*

In [1]:
%%bash
export JAVA_HOME=/soft/java
export PATH=$JAVA_HOME/bin:$PATH

#create a fasta file for our adapter
cat <<EOF > illumina_adapter.fasta
>TruSeq_Adapter_Index_11
GATCGGAAGAGCACACGTCTGAACTCCAGTCACGGCTACATCTCGTATGCCGTCTTCTGCTTG

EOF


mkdir trimmomatic
java -jar /projects_eg/projects/william/programs/Trimmomatic-0.35/trimmomatic-0.35.jar \
PE -phred33 s_cerevisiae_chrX_read1.fastq.gz s_cerevisiae_chrX_read2.fastq.gz \
trimmomatic/s_cerevisiae_chrX_read1_paired.fastq trimmomatic/s_cerevisiae_chrX_read1_unpaired.fastq \
trimmomatic/s_cerevisiae_chrX_read2_paired.fastq trimmomatic/s_cerevisiae_chrX_read2_unpaired.fastq \
ILLUMINACLIP:illumina_adapter.fasta:2:30:10:2:true LEADING:30 TRAILING:30 SLIDINGWINDOW:4:15 MINLEN:35



TrimmomaticPE: Started with arguments:
 -phred33 s_cere_reads_mapping_to_chrX_read1.fastq s_cere_reads_mapping_to_chrX_read2.fastq trimmomatic/s_cerevisiae_chrX_read1_paired.fastq trimmomatic/s_cerevisiae_chrX_read1_unpaired.fastq trimmomatic/s_cerevisiae_chrX_read2_paired.fastq trimmomatic/s_cerevisiae_chrX_read2_unpaired.fastq ILLUMINACLIP:illumina_adapter.fasta:2:30:10:2:true LEADING:30 TRAILING:30 SLIDINGWINDOW:4:15 MINLEN:35
Multiple cores found: Using 4 threads
Using Long Clipping Sequence: 'GATCGGAAGAGCACACGTCTGAACTCCAGTCACGGCTACATCTCGTATGCCGTCTTCTGCTTG'
ILLUMINACLIP: Using 0 prefix pairs, 1 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Input Read Pairs: 1284264 Both Surviving: 1284264 (100.00%) Forward Only Surviving: 0 (0.00%) Reverse Only Surviving: 0 (0.00%) Dropped: 0 (0.00%)
TrimmomaticPE: Completed successfully


# 4. Mapping RNAseq reads to the reference genome

First we will create a new .fastq file for chromosome 10 (Not for students to repeat, just creating the files here for convenience).

### Create an index
First we must create a Bowtie2 index using our reference genome (in this case we're just using chromosome 10) so that our reads can be mapped efficiently to the reference sequence. 

In [10]:
%%bash
export PATH=/soft/bio/sequence/bowtie2-2.2.4/:$PATH
# program    ||   reference genome     || output base filename
bowtie2-build  s_cerevisiae_chrX.fasta  s_cerevisiae_chrX



Settings:
  Output files: "s_cerevisiae_chr10.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  s_cerevisiae_chr10.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 186437
Using parameters --bmax 139828 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 139828 --dcv 1024
Constructing suffix-array element 

Building a SMALL index


### Map the reads with Bowtie2


In [2]:
%%bash
export PATH=/soft/bio/sequence/bowtie2-2.2.4/:$PATH
export BOWTIE2_INDEXES='.'

bowtie2 -x s_cerevisiae_chrX -p 2 \
-1 trimmomatic/s_cerevisiae_chrX_read1_paired.fastq \
-2 trimmomatic/s_cerevisiae_chrX_read2_paired.fastq \
-S 's_cerevisiae_chrX.sam'

1284264 reads; of these:
  1284264 (100.00%) were paired; of these:
    65833 (5.13%) aligned concordantly 0 times
    1158112 (90.18%) aligned concordantly exactly 1 time
    60319 (4.70%) aligned concordantly >1 times
    ----
    65833 pairs aligned concordantly 0 times; of these:
      20761 (31.54%) aligned discordantly 1 time
    ----
    45072 pairs aligned 0 times concordantly or discordantly; of these:
      90144 mates make up the pairs; of these:
        44158 (48.99%) aligned 0 times
        42685 (47.35%) aligned exactly 1 time
        3301 (3.66%) aligned >1 times
98.28% overall alignment rate


In [7]:
%%bash
export PATH=/soft/bio/sequence/samtools-0.1.18/:$PATH

### convert the .sam file into a .bam file and sort it
samtools view -Su s_cerevisiae_chrX.sam | samtools sort - s_cerevisiae_chrX_sorted
samtools index s_cerevisiae_chrX_sorted.bam

[samopen] SAM header is present: 1 sequences.
[bam_sort_core] merging from 2 files...




# 5. Assembling the mapped reads into a transcriptome with *Cufflinks*

In [8]:
%%bash
##Run Cufflinks2
##DESCRIPTION:transcript assembly (Cufflinks2)
mkdir cufflinks_output
### What do you want the files created by this script to be called? (short and descriptive)
export DIR_OF_OUTPUT_FILES="cufflinks_output"
export REFERENCE_GENOME="s_cerevisiae_chrX.fasta"
export REFERENCE_ANNOTATION="s_cerevisiae_chrX.gff"
export READ_1="trimmomatic/s_cerevisiae_chX_read1_paired.fastq"
export READ_2="trimmomatic/s_cerevisiae_chX_read2_paired.fastq"
export BAM_FILE="s_cerevisiae_chrX_sorted.bam"




##location of programs 
export PATH=/soft/bio/sequence/cufflinks-2.2.0/:$PATH

cufflinks $BAM_FILE -p 4 \
--frag-bias-correct $REFERENCE_GENOME \
-g $REFERENCE_ANNOTATION \
--max-bundle-frags 999999999 \
--max-bundle-length 4000000 \
--library-type fr-firststrand \
-o $DIR_OF_OUTPUT_FILES 


# cufflinks $BAM_FILE -p 4 --frag-bias-correct $REFERENCE_GENOME -g $REFERENCE_ANNOTATION --3-overhang-tolerance 120 --multi-read-correct --library-type fr-firststrand -o $DIR_OF_OUTPUT_FILES \
# --upper-quartile-norm --total-hits-norm --max-mle-iterations 6000 --max-bundle-frags 999999999 --frag-len-std-dev 86 --frag-len-mean 227 \
# --label $BASE_NAME_OF_OUTPUT_FILES --max-bundle-length 4000000 --min-frags-per-transfrag 20 --min-isoform-fraction .05 --min-intron-length 13 --max-intron-length 1001 --trim-3-avgcov-thresh 5 --trim-3-dropoff-frac 0.2 --max-multiread-fraction 0.33 --overlap-radius 1





You are using Cufflinks v2.2.1, which is the most recent release.
[14:52:10] Loading reference annotation.
[14:52:10] Inspecting reads and determining fragment length distribution.
> Processed 23 loci.                           [*************************] 100%
> Map Properties:
>	Normalized Map Mass: 1263164.00
>	Raw Map Mass: 1263164.00
>	Fragment Length Distribution: Truncated Gaussian (default)
>	              Default Mean: 200
>	           Default Std Dev: 80
[14:52:24] Assembling transcripts and estimating abundances.
> Processed 23 loci.                           [*************************] 100%
[14:54:11] Loading reference annotation and sequence.
[14:54:11] Learning bias parameters.
> Processed 292 loci.                          [*************************] 100%
[14:54:26] Re-estimating abundances with bias correction.
> Processed 292 loci.                          [*************************] 100%


# 6. etc